In [1]:
import pandas as pd
import psycopg2

In [2]:
import warnings
warnings.filterwarnings('ignore', message = '.*SQLAlchemy connectable')

In [3]:
%run connection2.ipynb import connection_sf 

In [4]:
connection = psycopg2.connect(
    dbname=connection_sf['DBNAME'],
    user=connection_sf['USER'],
    host=connection_sf['HOST'],
    password=connection_sf['PASSWORD'],
    port=connection_sf['PORT']
)

In [5]:
# функция для составления SQL запроса по введенным параметрам поиска вакансии  и вывода датафрейма с подходящим вакансиями

def vacancies_search():
    """finds matching vacancies from SQL database  

    Args:
        inputted requirements (str)

    Returns:
        displays 1 or 2 dataframe of vacancies      
    """
    
    print('Для увеличения количества вакансий можно пропускать поля для поиска, нажав Enter')
    print()

# form primary query
    query = f'''select 
                    v.name,
                    v.key_skills,
                    v.schedule,
                    v.employment,
                    v.experience,
                    v.salary_from,
                    v.salary_to,
                    a.name as region,
                    e.name as employer
                from public.vacancies as v
                    join public.areas as a on v.area_id = a.id
                    join public.employers e on v.employer_id = e.id
                where v.id = v.id
                '''
                
# input requirements      
    name = input('Наименование должности:')
    schedule = input('Выберите из предложенных варинтов график работы: Полный день, Удаленная работа, Гибкий график, Сменный график, Вахтовый метод')
    employment = input('Выберите из предложенных варинтов занятость: Полная занятость, Частичная занятость, Стажировка, Проектная работа')
    experience = input('Введите количество поных лет опыта:')
    city = input('Город:')
    big_city = input('Близлежащий крупный город:')
    salary = input('Желаемая зарплата:')
    skills = input('Укажите основные навыки ЧЕРЕЗ ЗАПЯТУЮ (например: Пользователь ПК, работа в команде, Python):')

# display inputted data for user   
    print('Вы ввели данные для поиска:')
    display(pd.DataFrame({
        'Название должности': [name],
        'График работы': [schedule],
        'Занятость': [employment],
        'Опыт, лет': [experience],
        'Желаемая зарплата, руб.': [salary],
        'Город': [city],
        'Крупный город рядом': [big_city],
        'Основные навыки': [skills]
    }))

# Add requirements to query if they are necessary    
    if len(name) > 0:
        name = name.strip()
        query += f''' and (lower(v.name) like '%{name.lower()}%')'''
    
    if len(schedule) > 0:
        schedule = schedule.strip()
        query += f''' and (lower(v.schedule) like '%{schedule.lower()}%')'''
    
    if len(employment) > 0:
        employment = employment.strip()
        query += f''' and (lower(v.employment) like '%{employment.lower()}%')'''
    
    if len(experience) > 0:
        if int(experience) < 1:
            query += f''' and (v.experience = 'Нет опыта')'''
        elif int(experience) < 3:
            query += f''' and (v.experience = 'От 1 года до 3 лет')'''
        elif int(experience) < 6:
            query += f''' and (v.experience = 'От 3 до 6 лет')'''
        elif int(experience) >= 6:
            query += f''' and (v.experience = 'Более 6 лет')'''
    
    if len(salary) > 0:
        query += f''' and ((v.salary_from <= {int(salary)} and v.salary_to >= {int(salary)})
                      or (v.salary_from <= {int(salary)} and v.salary_to is null)
                      or (v.salary_from is null and v.salary_to >= {int(salary)})
                      or (v.salary_from is null and v.salary_to is null))'''
    
    if len(skills) > 0:
        skills_list = skills.split(',')
        query += f''' and ('''
        for skill in skills_list:
            skill = skill.strip()
            query += f'''(lower(v.key_skills) like '%{skill.lower()}%') or '''
        query += f'''(v.key_skills is null))'''
        
    if len(big_city) > 0: # form second query for vacancies in nearby bigger city
        big_city = big_city.strip()
        query_big_city = query + f''' and (lower(a.name) = '{big_city.lower()}')'''
        
    if len(city) > 0:
        city = city.strip()
        query += f''' and (lower(a.name) = '{city.lower()}')'''
    
# add order of output       
    query += f'''\n order by v.salary_to desc nulls last, v.salary_from desc nulls last'''

# display dataframe/s 
    df_city = pd.read_sql_query(query, connection)
    print()
    print(f'По вашему запросу найдено вакансий: {df_city.shape[0]}')
    display(df_city)
    print()
    if len(big_city) > 0:
        query_big_city += f'''\n order by v.salary_to desc nulls last, v.salary_from desc nulls last'''
        df_big_city = pd.read_sql_query(query_big_city, connection)
        print(f'По вашему запросу в городе {big_city} найдено вакансий: {df_big_city.shape[0]}')
        display(df_big_city)
    

In [6]:
vacancies_search()

Для увеличения количества вакансий можно пропускать поля для поиска, нажав Enter

Вы ввели данные для поиска:


,Название должности,График работы,Занятость,"Опыт, лет","Желаемая зарплата, руб.",Город,Крупный город рядом,Основные навыки
0,Аналитик,полный день,,2,70000,Химки,МОСКВА,"SQL, Python"



По вашему запросу найдено вакансий: 2


,name,key_skills,schedule,employment,experience,salary_from,salary_to,region,employer
0,Аналитик,Python\tSQL\tАнглийский язык\tАнализ данных\tETL,Полный день,Полная занятость,От 1 года до 3 лет,None,None,Химки,Леруа Мерлен
1,Аналитик бизнес-процессов,Разработка технических заданий\tSQL\tPower BI\...,Полный день,Полная занятость,От 1 года до 3 лет,None,None,Химки,Агропромкомплектация



По вашему запросу в городе МОСКВА найдено вакансий: 381


,name,key_skills,schedule,employment,experience,salary_from,salary_to,region,employer
0,Системный аналитик (удаленно),Разработка технических заданий\tПостановка зад...,Полный день,Полная занятость,От 1 года до 3 лет,NaN,250000.0,Москва,Outlines Technologies
1,Аналитик IT,"SQL\tXML\tBPMN\tREST/SOAP\tJSON\tSmart Sheet, ...",Полный день,Полная занятость,От 1 года до 3 лет,NaN,230000.0,Москва,Экспириенс
2,Аналитик BI (Excel),None,Полный день,Полная занятость,От 1 года до 3 лет,NaN,220000.0,Москва,EXE.CTLY
3,Старший аналитик данных,Python\tSQL\tTableau\tPower BI\tАнализ данных,Полный день,Полная занятость,От 1 года до 3 лет,NaN,200000.0,Москва,ShinSale
4,Бизнес-аналитик,None,Полный день,Полная занятость,От 1 года до 3 лет,NaN,190000.0,Москва,РобоФинанс
...,...,...,...,...,...,...,...,...,...
376,Младший бизнес-аналитик,None,Полный день,Полная занятость,От 1 года до 3 лет,NaN,NaN,Москва,Ozon
377,Аналитик данных,SQL\tMS Excel,Полный день,Полная занятость,От 1 года до 3 лет,NaN,NaN,Москва,Hoff
378,Ведущий аналитик,None,Полный день,Полная занятость,От 1 года до 3 лет,NaN,NaN,Москва,PepsiCo
379,UX-писатель / Аналитик голосовых ботов,UX\tDiolog Flow\tSQL\tPython\tFigma\tРабота в ...,Полный день,Полная занятость,От 1 года до 3 лет,NaN,NaN,Москва,Voximplant
